In [15]:
import cv2
import numpy as np

# Chessboard pattern size (inner corners in width and height)
pattern_size = (6, 8)

# Real-world size of a square on the chessboard (e.g., in meters or millimeters)
square_size = 2.5
# Prepare object points
objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[1], 0:pattern_size[0]].T.reshape(-1, 2) 

# Arrays to store object points and image points
objpoints = []         # 3D points in real-world space
imgpoints_left = []    # 2D points in left image plane
imgpoints_right = []   # 2D points in right image plane


In [16]:
import os
import numpy as np
# Path to the folder containing the images
folder_path = "Left Camera Chess"

# List to store left image file paths
left_images = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if "l" in filename.lower():  # Check if "left" is in the filename
        left_images.append(os.path.join(folder_path, filename))

# Print the list of left images
print("Left Images:")
print(left_images)

Left Images:
['Left Camera Chess/chess8l.jpeg', 'Left Camera Chess/chess10l.jpeg', 'Left Camera Chess/chess9l.jpeg', 'Left Camera Chess/chess5l.jpeg', 'Left Camera Chess/chess7l.jpeg', 'Left Camera Chess/chess3l.jpeg', 'Left Camera Chess/chess1l.jpeg', 'Left Camera Chess/chess6l.jpeg', 'Left Camera Chess/chess4l.jpeg', 'Left Camera Chess/chess2l.jpeg']


In [17]:
folder_path = "Right Camera Chess"

# List to store left image file paths
right_images = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if "r" in filename.lower():  # Check if "left" is in the filename
        right_images.append(os.path.join(folder_path, filename))

# Print the list of left images
print("Right Images:")
print(right_images)

Right Images:
['Right Camera Chess/chess9r.jpeg', 'Right Camera Chess/chess10r.jpeg', 'Right Camera Chess/chess8r.jpeg', 'Right Camera Chess/chess2r.jpeg', 'Right Camera Chess/chess4r.jpeg', 'Right Camera Chess/chess6r.jpeg', 'Right Camera Chess/chess1r.jpeg', 'Right Camera Chess/chess3r.jpeg', 'Right Camera Chess/chess7r.jpeg', 'Right Camera Chess/chess5r.jpeg']


In [18]:
for left_path, right_path in zip(left_images, right_images):
    img_left = cv2.imread(left_path, cv2.IMREAD_GRAYSCALE)
    img_right = cv2.imread(right_path, cv2.IMREAD_GRAYSCALE)

    # Detect chessboard corners in both images
    ret_left, corners_left = cv2.findChessboardCorners(img_left, pattern_size)
    ret_right, corners_right = cv2.findChessboardCorners(img_right, pattern_size)

    if ret_left and ret_right:
        # Refine corner detection
        corners_left = cv2.cornerSubPix(img_left, corners_left, (11, 11), (-1, -1), 
                                        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        corners_right = cv2.cornerSubPix(img_right, corners_right, (11, 11), (-1, -1), 
                                         criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))

        # Append points
    objpoints.append(objp)
    imgpoints_left.append(corners_left)
    imgpoints_right.append(corners_right)

In [19]:
# Calibrate the left camera
ret_left, mtx_left, dist_left, rvecs_left, tvecs_left = cv2.calibrateCamera(
    objpoints, imgpoints_left, img_left.shape[::-1], None, None
)

# Calibrate the right camera
ret_right, mtx_right, dist_right, rvecs_right, tvecs_right = cv2.calibrateCamera(
    objpoints, imgpoints_right, img_right.shape[::-1], None, None
)

# Print intrinsics for verification
print("Left Camera Matrix:\n", mtx_left)
print("Right Camera Matrix:\n", mtx_right)



Left Camera Matrix:
 [[3.55919524e+03 0.00000000e+00 5.36908639e+02]
 [0.00000000e+00 3.63513511e+03 9.29793444e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Right Camera Matrix:
 [[3.93855310e+03 0.00000000e+00 4.26410801e+02]
 [0.00000000e+00 6.01968364e+03 6.42043524e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [20]:
# Stereo calibration
flags = cv2.CALIB_FIX_INTRINSIC  # Use this if intrinsics are precomputed
ret, mtx_left, dist_left, mtx_right, dist_right, R, T, E, F = cv2.stereoCalibrate(
    objpoints, imgpoints_left, imgpoints_right,
    mtx_left, dist_left, mtx_right, dist_right,
    img_left.shape[::-1], flags=flags
)

# Print extrinsic parameters
print("Rotation Matrix (R):\n", R)
print("Translation Vector (T):\n", T)


Rotation Matrix (R):
 [[ 0.98164755 -0.18962039 -0.02030243]
 [ 0.19051132  0.97029521  0.14910612]
 [-0.00857421 -0.1502375   0.98861275]]
Translation Vector (T):
 [[-162.28105836]
 [  12.54724592]
 [5750.75871017]]


In [27]:
# Stereo rectification
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
    mtx_left, dist_left, mtx_right, dist_right, img_left.shape[::-1], R, T
)

# Save parameters for future use
np.savez("stereo_calibration.npz", 
         mtx_left=mtx_left, dist_left=dist_left, 
         mtx_right=mtx_right, dist_right=dist_right, 
         R=R, T=T, Q=Q)

print(Q)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  3.15978279e+01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.42673712e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.74887417e+03]
 [ 0.00000000e+00  0.00000000e+00  1.73820491e-04 -0.00000000e+00]]


In [28]:
calibration_data = np.load("stereo_calibration.npz")
mtx_left = calibration_data['mtx_left']
dist_left = calibration_data['dist_left']
mtx_right = calibration_data['mtx_right']
dist_right = calibration_data['dist_right']
R = calibration_data['R']
T = calibration_data['T']
Q = calibration_data['Q']
print(Q)

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  3.15978279e+01]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -1.42673712e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.74887417e+03]
 [ 0.00000000e+00  0.00000000e+00  1.73820491e-04 -0.00000000e+00]]


In [23]:
# Load a pair of scene images
img_left = cv2.imread("Left Camera Scene/scene1l.jpeg", cv2.IMREAD_GRAYSCALE)
img_right = cv2.imread("Right Camera Scene/scene1r.jpeg", cv2.IMREAD_GRAYSCALE)
# print(img_left)
# print(img_right)

# Stereo rectification
image_size = img_left.shape[::-1]
R1, R2, P1, P2, Q, _, _ = cv2.stereoRectify(
    mtx_left, dist_left, mtx_right, dist_right, image_size, R, T
)

# Compute rectification maps
map1_left, map2_left = cv2.initUndistortRectifyMap(
    mtx_left, dist_left, R1, P1, image_size, cv2.CV_16SC2
)
map1_right, map2_right = cv2.initUndistortRectifyMap(
    mtx_right, dist_right, R2, P2, image_size, cv2.CV_16SC2
)
print(map1_right)
print(map2_right)
print(img_right)
# Rectify images
rectified_left = cv2.remap(img_left, map1_left, map2_left, cv2.INTER_LINEAR)
rectified_right = cv2.remap(img_right, map1_right, map2_right, cv2.INTER_LINEAR)
print(rectified_left)



[[[-1196 13785]
  [-1194 13782]
  [-1191 13780]
  ...
  [  891 14172]
  [  893 14175]
  [  896 14178]]

 [[-1197 13792]
  [-1195 13790]
  [-1192 13787]
  ...
  [  891 14179]
  [  894 14183]
  [  896 14186]]

 [[-1198 13800]
  [-1195 13797]
  [-1193 13795]
  ...
  [  892 14187]
  [  894 14191]
  [  897 14194]]

 ...

 [[-3206 32767]
  [-3200 32767]
  [-3193 32767]
  ...
  [ 2099 32767]
  [ 2106 32767]
  [ 2112 32767]]

 [[-3208 32767]
  [-3202 32767]
  [-3195 32767]
  ...
  [ 2100 32767]
  [ 2107 32767]
  [ 2113 32767]]

 [[-3210 32767]
  [-3204 32767]
  [-3197 32767]
  ...
  [ 2102 32767]
  [ 2108 32767]
  [ 2115 32767]]]
[[ 352  916  456 ...  167  599  998]
 [ 711  251  815 ...  948  323  755]
 [  47  611  151 ...  705  112  512]
 ...
 [ 548  306   33 ...  373  676   52]
 [  37  820  546 ...  318  654   30]
 [ 550  309   68 ...  296  664 1000]]
[[212 212 212 ... 196 196 196]
 [212 212 212 ... 196 196 196]
 [212 212 212 ... 196 196 196]
 ...
 [201 200 199 ... 234 234 234]
 [201 201 200

In [29]:
stereo = cv2.StereoBM_create(numDisparities=16*3, blockSize=15)
# Compute disparity map
disparity = stereo.compute(rectified_left, rectified_right).astype(np.float32) / 16.0
print(disparity)

# Normalize the disparity map for visualization
disparity_normalized = cv2.normalize(disparity, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)


[[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]]


: 

In [25]:
# Reproject disparity to 3D points
points_3D = cv2.reprojectImageTo3D(disparity, Q)

# Mask invalid points
mask = disparity > disparity.min()
points_3D = points_3D[mask]

# Get the color information from the left image
colors = cv2.cvtColor(cv2.imread("Left Camera Scene/scene1l.jpeg"), cv2.COLOR_BGR2RGB)[mask]

# Save as a PLY file for 3D visualization
def save_point_cloud(filename, points, colors):
    with open(filename, 'w') as f:
        f.write("ply\nformat ascii 1.0\n")
        f.write(f"element vertex {len(points)}\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        f.write("property uchar red\nproperty uchar green\nproperty uchar blue\n")
        f.write("end_header\n")
        for p, c in zip(points, colors):
            f.write(f"{p[0]} {p[1]} {p[2]} {c[0]} {c[1]} {c[2]}\n")

save_point_cloud("scene_reconstruction.ply", points_3D, colors)


In [26]:
import open3d as o3d

pcd = o3d.io.read_point_cloud("scene_reconstruction.ply")

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])


[Open3D WARNING] Read PLY failed: number of vertex <= 0.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
